In [1]:
%%capture
!pip3 install seaborn

from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from scipy.spatial.distance import euclidean
from math import inf


import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from sklearn.cluster import MeanShift


In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [3]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = ['.', ',', '!', '?', ';', ':']

def remove_stopwords(text):
    filtered_text = [w for w in text.split() if not w in stop_words]
    return filtered_text

def remove_punctuation(text):
    filtered_text = ''.join([w for w in text if w not in punctuation])
    return filtered_text

def map_contractions(text):
    filtered_text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    return filtered_text

In [5]:
tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w) for w in text.split()])
    return lemmatized_text

def word_stemmer(text):
    stemmed_text = PorterStemmer(text)
    return stemmed_text

In [6]:
def clean_text(text):
    cleaned_text = text.lower()
    cleaned_text = map_contractions(cleaned_text)
    #cleaned_text = word_stemmer(cleaned_text)
    #cleaned_text = word_lemmatizer(cleaned_text)
    return cleaned_text

In [7]:
word_lemmatizer("was")


'wa'

In [8]:
doc = "She reached her goal, exhausted. Even more chilling to her was that the euphoria that she thought she'd feel upon reaching it wasn't there. Something wasn't right. Was this the only feeling she'd have for over five years of hard work?"
cleaned_text = clean_text(doc)
cleaned_sentences = sent_tokenize(cleaned_text)
cleaned_sentences

['she reached her goal, exhausted.',
 'even more chilling to her was that the euphoria that she thought she would feel upon reaching it was not there.',
 'something was not right.',
 'was this the only feeling she would have for over five years of hard work?']

In [9]:
logging.set_verbosity(logging.ERROR)

vectors = embed(cleaned_sentences)

In [10]:
vectors

<tf.Tensor: shape=(4, 512), dtype=float32, numpy=
array([[-0.00973231, -0.02009696,  0.04489999, ...,  0.02190253,
        -0.01674248, -0.0298488 ],
       [ 0.02847688,  0.05883124, -0.01837788, ..., -0.0388983 ,
         0.02044887,  0.03118455],
       [-0.09053303,  0.03494389,  0.04732297, ..., -0.05597072,
         0.01310057, -0.00303079],
       [ 0.0005601 , -0.06925794, -0.00274093, ..., -0.07330345,
         0.00431386, -0.01050149]], dtype=float32)>

In [11]:
clusters = MeanShift().fit(vectors)
labels = clusters.labels_
centers = clusters.cluster_centers_

In [12]:
labels

array([2, 0, 3, 1], dtype=int64)

In [13]:
keep_labels = []

for i in range(len(centers)):
    indeces = np.where(labels == i)
    most_centered = indeces[0][0]
    if len(indeces[0]) == 1:
        keep_labels.append(most_centered)
    else:
        for j in range(len(indeces)):
            shortest_distance = inf
            distance = euclidean(centers[labels[i]], vectors[indeces[0][j]])
            if distance < shortest_distance:
                shortest_distance = distance
                most_centered = indeces[0][j]
        keep_labels.append(most_centered)
        
keep_labels

[1, 3, 0, 2]

In [14]:
keep_sentences = []

for i in range(len(cleaned_sentences)):
    if i in keep_labels:
        keep_sentences.append(cleaned_sentences[i])

summarized_text = ' '.join(sentence for sentence in keep_sentences)   

summarized_text

'she reached her goal, exhausted. even more chilling to her was that the euphoria that she thought she would feel upon reaching it was not there. something was not right. was this the only feeling she would have for over five years of hard work?'